## Import all the needed packages

In [68]:
# DataFrame
import pandas as pd

# Elasticsearch
from elasticsearch import Elasticsearch, helpers

from datetime import datetime

import matplotlib.pyplot as plt

# isnan()
import math

## Create the local raw_data_pool

In [2]:
raw_data_pool = {}

def make_sure(src_site, dest_site):
    if src_site not in raw_data_pool:
        raw_data_pool[src_site] = {}
    if dest_site not in raw_data_pool[src_site]:
        raw_data_pool[src_site][dest_site] = pd.DataFrame()

def put_data(src_site, dest_site, timestamp_epoch, column_type, value):
    make_sure(src_site, dest_site)
    raw_data_pool[src_site][dest_site].set_value(timestamp_epoch, column_type, value)

## Create the elasticsearch connection

In [3]:
es = Elasticsearch(['atlas-kibana.mwt2.org:9200'])
my_index = "network_weather_2-*"
my_query = {}
my_timeout = 600

In [4]:
# my_query = \
# {
#   "size": 100,
#   "sort": [
#     {
#       "timestamp": {
#         "order": "desc",
#         "unmapped_type": "boolean"
#       }
#     }
#   ],
#   "query": {
#     "filtered": {
#       "query": {
#         "query_string": {
#           "analyze_wildcard": True,
#           "lowercase_expanded_terms": False,
#           "query": "*"
#         }
#       },
#       "filter": {
#         "bool": {
#           "must": [
#             {
#               "query": {
#                 "match": {
#                   "srcSite": {
#                     "query": "Australia-ATLAS",
#                     "type": "phrase"
#                   }
#                 }
#               }
#             },
#             {
#               "query": {
#                 "match": {
#                   "destSite": {
#                     "query": "SFU-LCG2",
#                     "type": "phrase"
#                   }
#                 }
#               }
#             },
#             {
#               "range": {
#                 "timestamp": {
#                   "gte": 1454306400000,
#                   "lte": 1456812000000,
#                   "format": "epoch_millis"
#                 }
#               }
#             }
#           ],
#           "must_not": []
#         }
#       }
#     }
#   },
#   "fields": [
#     "*",
#     "_source"
#   ],
#   "script_fields": {
#     "throughput [Gbps]": {
#       "script": "doc['throughput'].value/1073741824.0",
#       "lang": "expression"
#     }
#   },
#   "fielddata_fields": [
#     "timestamp"
#   ]
# }

In [5]:
# my_query

In [6]:
# type(my_query)

In [7]:
# my_query["query"]["filtered"]["filter"]

In [8]:
# response = es.search(index=my_index, body=my_query, request_timeout=600)

In [9]:
# response

In [10]:
# def ask_total_hits(query_body)

In [11]:
# def generate_query(src_site, dest_site, start_timestamp, end_timestamp):
#     

In [12]:
# # https://gist.github.com/drorata/146ce50807d16fd4a6aa

# # Initialize the scroll
# page = es.search(
#   index = 'network_weather_2-*',
#   doc_type = 'latency',
#   scroll = '2m',
#   search_type = 'scan',
#   size = 10000,
#   body = my_query)

# sid = page['_scroll_id']
# scroll_size = page['hits']['total']
  
# # Start scrolling
# while (scroll_size > 0):
#     print("Scrolling...")
#     page = es.scroll(scroll_id = sid, scroll = '2m')
#     # Update the scroll ID
#     sid = page['_scroll_id']
#     # Get the number of results that we returned in the last scroll
#     scroll_size = len(page['hits']['hits'])
# #     print(page['hits'])
#     print("scroll size: " + str(scroll_size))
#     # Do something with the obtained page

In [13]:
# my_query = \
# {
    
# }

In [14]:
# scroll = helpers.scan(client=es, query=my_query)
# count = 0

# for res in scroll:
#     count += 1
# #     print(res)
# #     print("===============")

# print(count)

In [15]:
src_site = "Australia-ATLAS"
dest_site = "SFU-LCG2"
# timestamp = { 'gte': 1454306400000, 'lte': 1456812000000, 'format': 'epoch_millis' }
# timestamp = { 'gte': 1454306400000, 'lt': 1456812000000 }
# timestamp = { 'gte': '2016-02-01T00:00:00Z', 'lt': '2016-03-01T00:00:00Z' }
timestamp = { 'gte': '2016-01-01T00:00:00Z', 'lt': '2016-06-01T00:00:00Z' }

my_query = {}
my_query['query'] = {}
my_query['query']['filtered'] = {}
my_query['query']['filtered']['query'] = { "match_all": {} }
my_query['query']['filtered']['filter'] = {}
my_query['query']['filtered']['filter']['bool'] = {}
my_query['query']['filtered']['filter']['bool']['must'] = []
my_query['query']['filtered']['filter']['bool']['must'].append({ 'term': { 'srcSite': src_site } })
my_query['query']['filtered']['filter']['bool']['must'].append({ 'term': { 'destSite': dest_site } })
my_query['query']['filtered']['filter']['bool']['must'].append({ 'range': { 'timestamp': timestamp } })
my_query['fielddata_fields'] = [ 'timestamp' ]

my_query

{'fielddata_fields': ['timestamp'],
 'query': {'filtered': {'filter': {'bool': {'must': [{'term': {'srcSite': 'Australia-ATLAS'}},
      {'term': {'destSite': 'SFU-LCG2'}},
      {'range': {'timestamp': {'gte': '2016-01-01T00:00:00Z',
         'lt': '2016-06-01T00:00:00Z'}}}]}},
   'query': {'match_all': {}}}}}

In [16]:
scroll = list(helpers.scan(client=es, query=my_query))
count = 0

for res in scroll:
    count += 1
#     print(res)
#     print("====================")
    column_type = res['_type']
    timestamp_epoch = res['fields']['timestamp'][0]
    if column_type == 'latency':
        put_data(src_site, dest_site, timestamp_epoch, 'iso_8601', res['_source']['timestamp'])
        put_data(src_site, dest_site, timestamp_epoch, 'delay_median', res['_source']['delay_median'])
        put_data(src_site, dest_site, timestamp_epoch, 'delay_mean', res['_source']['delay_mean'])
        put_data(src_site, dest_site, timestamp_epoch, 'delay_sd', res['_source']['delay_sd'])
#         print(res['_source']['delay_median'])
#         print(res['_source']['delay_mean'])
#         print(res['_source']['delay_sd'])
#         print(res['fields']['timestamp'][0])
#         print(res['_source']['timestamp'])
    elif column_type == 'packet_loss_rate':
        put_data(src_site, dest_site, timestamp_epoch, 'iso_8601', res['_source']['timestamp'])
        put_data(src_site, dest_site, timestamp_epoch, 'packet_loss', res['_source']['packet_loss'])
#         print(res['_source']['packet_loss'])
#         print(res['fields']['timestamp'][0])
#         print(res['_source']['timestamp'])
    elif column_type == 'throughput':
        put_data(src_site, dest_site, timestamp_epoch, 'iso_8601', res['_source']['timestamp'])
        put_data(src_site, dest_site, timestamp_epoch, 'throughput', res['_source']['throughput'])
#         print(res['_source']['throughput'])
#         print(res['fields']['timestamp'][0])
#         print(res['_source']['timestamp'])
    else:
        print("Error: type should be latency, packet_loss_rate, or throughput. ")
    
#     if count >= 111:
#         break;

print(count)

114721


In [17]:
# timestamp = "2016-02-01T07:05:00"
# # datetime.strptime(timestamp, "%Y-%m-%dT%H:%M:%S.%fZ")
# a = datetime.strptime(timestamp, "%Y-%m-%dT%H:%M:%S")

In [18]:
# type(a)

In [19]:
# put_data("aaa", "bbb", a, "latency", 898.3)

In [20]:
# raw_data_pool['aaa']['bbb']

In [21]:
a = raw_data_pool[src_site][dest_site]

In [22]:
a.count(axis='index')

iso_8601        3219
packet_loss     3177
delay_median    3105
delay_mean      3105
delay_sd        3105
throughput        21
dtype: int64

In [23]:
# a.plot()

In [24]:
a

,iso_8601,packet_loss,delay_median,delay_mean,delay_sd,throughput
1451612100000,2016-01-01T01:35:00,0.0,84.4,84.447333,0.053786,NaN
1451614200000,2016-01-01T02:10:00,NaN,85.6,85.584200,0.092288,NaN
1451614500000,2016-01-01T02:15:00,0.0,NaN,NaN,NaN,NaN
1451616900000,2016-01-01T02:55:00,0.0,86.7,86.706333,0.050595,NaN
1452388500000,2016-01-10T01:15:00,0.0,81.3,81.277556,0.064778,NaN
1452390600000,2016-01-10T01:50:00,0.0,80.3,80.300167,0.043011,NaN
1452392400000,2016-01-10T02:20:00,0.0,83.0,83.004208,0.153796,NaN
1452394500000,2016-01-10T02:55:00,0.0,84.1,84.077417,0.147530,NaN
1452396600000,2016-01-10T03:30:00,0.0,83.7,83.706333,0.232149,NaN
1452547500000,2016-01-11T21:25:00,0.0,81.8,81.764667,0.052772,NaN


In [25]:
a.to_pickle("raw_data_1")

In [39]:
a.sort_index(inplace=True)

In [26]:
# tp = a.loc[a['throughput'] != NaN]

NameError: name 'NaN' is not defined

In [40]:
t = a[a.throughput.notnull()]

In [41]:
t

,iso_8601,packet_loss,delay_median,delay_mean,delay_sd,throughput
1451675090000,2016-01-01T19:04:50,NaN,NaN,NaN,NaN,105920000.0
1451805042000,2016-01-03T07:10:42,NaN,NaN,NaN,NaN,606593000.0
1452076266000,2016-01-06T10:31:06,NaN,NaN,NaN,NaN,625393000.0
1452158122000,2016-01-07T09:15:22,NaN,NaN,NaN,NaN,460065000.0
1452661247000,2016-01-13T05:00:47,NaN,NaN,NaN,NaN,527882000.0
1452753236000,2016-01-14T06:33:56,NaN,NaN,NaN,NaN,308804000.0
1452794779000,2016-01-14T18:06:19,NaN,NaN,NaN,NaN,239490000.0
1453975840000,2016-01-28T10:10:40,NaN,NaN,NaN,NaN,552811000.0
1454047943000,2016-01-29T06:12:23,NaN,NaN,NaN,NaN,167993000.0
1454706290000,2016-02-05T21:04:50,NaN,NaN,NaN,NaN,681488000.0


In [43]:
t.shape

(21, 6)

In [45]:
# print(t.iloc(0))

In [48]:
# tt = t.iloc(0)['index']

TypeError: cannot do positional indexing on <class 'pandas.indexes.numeric.Int64Index'> with these indexers [index] of <class 'str'>

In [86]:
dataframe_size = a.shape
dataframe_size
p = pd.DataFrame()
for r in range(dataframe_size[0]):
    if not math.isnan(a.iloc[r]['throughput']):
        if (r != 0) and (r != dataframe_size[0] - 1):
            p = p.append(a.iloc[r-1])
            p = p.append(a.iloc[r])
            p = p.append(a.iloc[r+1])

In [87]:
p

,delay_mean,delay_median,delay_sd,iso_8601,packet_loss,throughput
1451674500000,84.436417,84.4,1.625300,2016-01-01T18:55:00,0.0,NaN
1451675090000,NaN,NaN,NaN,2016-01-01T19:04:50,NaN,105920000.0
1451676600000,83.418900,83.4,0.112425,2016-01-01T19:30:00,0.0,NaN
1451803200000,76.117833,76.1,0.087742,2016-01-03T06:40:00,0.0,NaN
1451805042000,NaN,NaN,NaN,2016-01-03T07:10:42,NaN,606593000.0
1451805600000,83.026250,83.0,0.049016,2016-01-03T07:20:00,0.0,NaN
1452072900000,83.264583,83.0,1.278601,2016-01-06T09:35:00,0.0,NaN
1452076266000,NaN,NaN,NaN,2016-01-06T10:31:06,NaN,625393000.0
1452076800000,83.504767,83.8,0.435217,2016-01-06T10:40:00,0.0,NaN
1452157200000,82.699056,82.4,1.656276,2016-01-07T09:00:00,0.0,NaN
